In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gbm

#### A few starting parameters... (*none of these have been used yet*)

In [4]:
# The days from which we calculate drift (mu) and diffusion (sigma)
startday = 800   # '0' corresponds to 2015-02-01, it counts up from there
endday = 1108

N = 100    # number of days to predict

#### Reading in & Formatting Price Data
The .csv read from contains the price of [bitcoin](https://bitcoin.org/en/) from 2015-02-01 to 2018-02-13.

In [5]:
prices_raw = pd.read_csv('data/price_data.csv', infer_datetime_format=True)
p = prices_raw['price']
print(p.head(2))
print(p.tail(2))

0    226.3959
1    237.5409
Name: price, dtype: float64
1107    8891.2125
1108    8516.2438
Name: price, dtype: float64


#### Reading in & Formatting Quantified Sentiment
The .csv we read from here contains daily sentiment scores from 2015-02-01 to 2018-02-13 for the title of every post in the [Bitcoin subreddit](https://www.reddit.com/r/Bitcoin/), an online Bitcoin community.

There seems to be *a lot* of noise in the form of 0's (neutral sentiment). **How can this effectively be handled?**

For now, I will take the daily average.

In [6]:
sentiment_raw = pd.read_csv('data/reddit_sentiment_data.csv', infer_datetime_format=True)
s = sentiment_raw.mean(axis=1)
print(s.head(2))
print(s.tail(2))

0    0.068356
1    0.068356
dtype: float64
1107    0.046143
1108    0.004784
dtype: float64


#### Calculating dS as predicted by [Geometric Brownian Motion](https://en.wikipedia.org/wiki/Geometric_Brownian_motion), where S = Price.

In [78]:
# def GBM_step(mu, S, N, sigma, W)